In [30]:
!git clone https://github.com/Samudragupta123/sentiment_project.git

fatal: destination path 'sentiment_project' already exists and is not an empty directory.


In [31]:
import os
print(os.listdir())

['.config', 'sentiment_project', 'sample_data']


In [32]:
print(os.listdir('sentiment_project/data'))

['raw_dataset.csv']


In [33]:
import pandas as pd
import numpy as np
dataset=pd.read_csv('sentiment_project/data/raw_dataset.csv')
dataset.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [34]:
#dropping redundant columns
dataset=dataset.drop(dataset.columns[[0,6,7,8,9,11,12,13,14]], axis=1)

In [35]:
dataset.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text
0,neutral,1.0000,NaN,NaN,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,0.3486,NaN,0.0000,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,NaN,NaN,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...


In [36]:
datasetfinal=dataset.fillna(0)

In [37]:
datasetfinal.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text
0,neutral,1.0000,0,0.0000,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,0.3486,0,0.0000,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,0,0.0000,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,@VirginAmerica and it's a really big bad thing...


In [38]:
X=dataset.iloc[:,1:5].values
y=dataset.iloc[:,0].values

In [39]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [45]:
import torch
import numpy as np
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, idx):
        # Get one sample from the numpy arrays
        sample_features = self.x_data[idx]
        sample_label = self.y_data[idx]
        return sample_features, sample_label
train_dataset= MyDataset(X_train, y_train)
test_dataset= MyDataset(X_test, y_test)

In [46]:
from torch.utils.data import DataLoader

# Training loader: Shuffling is True
train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=1000, 
    shuffle=True  
)

# Testing/Validation loader: Shuffling is usually False
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=100, 
    shuffle=False
)


In [47]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download required packages
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [48]:
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download required data for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [49]:
def nltk_preprocess(text, mode='lemmatize'):
    if not text or text == 'nan': # Handle empty/null values from your CSV
        return ""

  # Tokenization and basic cleaning (lowercase, stopword removal)
    tokens = word_tokenize(str(text).lower())
    tokens = [w for w in tokens if w not in stop_words and w not in string.punctuation]
    
    if mode == 'lemmatize':
        # Reduces to dictionary form: "cats" -> "cat", "feet" -> "foot"
        cleaned_tokens = [lemmatizer.lemmatize(w) for w in tokens]
    elif mode == 'stem':
        # Chops off suffixes: "running" -> "run", "likely" -> "like"
        cleaned_tokens = [stemmer.stem(w) for w in tokens]
    
    return " ".join(cleaned_tokens)


In [50]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Required for POS tagging
nltk.download('averaged_perceptron_tagger_eng') 
nltk.download('wordnet')
def get_wordnet_pos(treebank_tag):
    """Maps Penn Treebank tags to WordNet constants."""
    if treebank_tag.startswith('J'): return wordnet.ADJ
    elif treebank_tag.startswith('V'): return wordnet.VERB
    elif treebank_tag.startswith('N'): return wordnet.NOUN
    elif treebank_tag.startswith('R'): return wordnet.ADV
    return wordnet.NOUN


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
lemmatizer = WordNetLemmatizer()

def process_with_pos(text):
    tokens = nltk.word_tokenize(str(text).lower())
    pos_tags = nltk.pos_tag(tokens) 
    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in pos_tags
    ]
    return " ".join(lemmatized)


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_corpus = datasetfinal['text'].astype(str)
reason_corpus = datasetfinal['negativereason'].astype(str)

# Combine both fields to build vocabulary
all_text = pd.concat([text_corpus, reason_corpus], axis=0)

tfidf = TfidfVectorizer(max_features=5000)  # keep fixed size
tfidf.fit(all_text)

TfidfVectorizer(max_features=5000)

In [59]:
from sklearn.preprocessing import OneHotEncoder
import torch
import numpy as np

# Pre-fit encoders on your original dataframe columns
# sentiment_enc: for column 0 (y)
# airline_enc: for column 4 (in x_batch it's index 3)
sentiment_enc = OneHotEncoder(sparse_output=False).fit(dataset[['airline_sentiment']])
airline_enc = OneHotEncoder(sparse_output=False).fit(dataset[['airline']])

def custom_collate(batch):
    # Separate the batch into features (x) and targets (y)
    x_batch_raw, y_batch_raw = zip(*batch)
    
    # --- 1. One-Hot Encoding for Target (y_batch) ---
    # Column 0: airline_sentiment
    y_encoded = torch.tensor(sentiment_enc.transform(np.array(y_batch_raw).reshape(-1, 1)), dtype=torch.float32)

    # --- 2. Processing x_batch (Columns 1-5) ---
    # Col indices in x_batch: 0: airline_sentiment_conf, 1: negativereason, 
    # 2: negativereason_conf, 3: airline, 4: text
    
    # A. One-Hot Encoding: 'airline' (Index 3)
    airlines = [[row[3]] for row in x_batch_raw]
    airline_ohe = torch.tensor(airline_enc.transform(airlines), dtype=torch.float32)
    
    # B. NLP: 'negativereason' (Index 1) and 'text' (Index 4)
    def safe_text(x):
        if x is None:
            return ""
        if isinstance(x, float):   # catches NaN
            return ""
        return str(x)

    processed_reason = [
        process_with_pos(nltk_preprocess(safe_text(row[1])))
        for row in x_batch_raw
    ]

    processed_text = [
        process_with_pos(nltk_preprocess(safe_text(row[4])))
        for row in x_batch_raw
    ]
    
    # C. Numeric: Confidence scores (Indices 0 and 2)
    numeric_feats = torch.tensor([[row[0], row[2]] for row in x_batch_raw], dtype=torch.float32)

    text_vec = torch.tensor(
    tfidf.transform([safe_text(t) for t in processed_text]).toarray(),
    dtype=torch.float32
)

    reason_vec = torch.tensor(
    tfidf.transform([safe_text(t) for t in processed_reason]).toarray(),
    dtype=torch.float32
)

    # Combine all processed data into a single dictionary
    return {
        "numeric_and_ohe": torch.cat([numeric_feats, airline_ohe], dim=1),
        "text_data": text_vec,
        "reason_data": reason_vec,
        "targets": y_encoded
    }


In [60]:
train_loader.collate_fn = custom_collate
test_loader.collate_fn = custom_collate